In [15]:
from bs4 import BeautifulSoup
from dateutil.parser import parse
import requests
import pandas as pd
import os
import re
import json
from tqdm import tqdm


In [2]:
url="https://www.aljazeera.com/where/mozambique/"
link=[]
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
#print(soup)

In [19]:
from tqdm import tqdm
from time import sleep
 
 
for i in tqdm(range(0, 100), initial = 50,
              desc ="Text You Want"):
    sleep(.1)

Text You Want: 150it [00:10,  9.75it/s]


In [20]:
for i in tqdm(range(0, 100), initial = 50,
              desc ="Web Scraping"):
    news_web= soup.find_all("h3", attrs={"class": "gc__title"})
    news_dates=soup.find_all("div",attrs={"class": "date-simple css-1yjq2zp"})
    news_df=pd.DataFrame(columns=["article_link", "article_title", "published_date"])
    i=0
    for news in news_web:
        data_dict = {"article_link": [news.a["href"]],"article_title": [news.text.encode("ascii", "ignore")],  "published_date": [parse(news_dates[i].select("span")[1].text)]}
        df = pd.DataFrame (data_dict)
        news_df= pd.concat([news_df, df], ignore_index=True)
        i+=1
    news_df=news_df.sort_values(by="published_date",ascending=False)[:10]   

Web Scraping: 150it [00:01, 55.37it/s]


In [23]:
articles=[]
i=0
for link in news_df["article_link"]:
    html_content = requests.get("https://www.aljazeera.com"+link).text
    soup = BeautifulSoup(html_content, "lxml")
    article_web=soup.find("div",attrs={"class": re.compile("wysiwyg wysiwyg--all-content css-[a-z0-9]")})
    s=""
    for p in article_web.select("p"):
        s+=p.text
    articles.append(s)
    i+=1
news_df["article"]=articles  

In [5]:
news_df_json=news_df.drop(["article_link","published_date"], axis = 1)
path= os.path.abspath(os.getcwd())+"/articles.json"
with open('articles.json', 'w', encoding='utf-8') as file:
    news_df_json.to_json(file,orient="records", force_ascii=False)

In [6]:
f= open(path,"r")
f1=json.load(f)
print(f1)

[{'article_title': 'Floods hit South Africas KwaZulu-Natal province again', 'article': 'Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast, according to officials, only a month after the worst floods in living memory killed more than 400.\nThe floods over the weekend hit infrastructure, roads, bridges and damaged buildings, mainly in the port city of Durban, the capital of KwaZulu-Natal province and home to some 3.9 million people.“Some areas are inaccessible and have become islands at this stage,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday.\nNo fatalities were immediately recorded, but Zikalala added: “We are yet to receive a full comprehensive report on the impact of these latest heavy rains, but as reports come in a worrying picture is however emerging.”\nIn April, torrential rain led to floods, landslides and the death of 435 people in KwaZulu-Natal and authorities said survivors were left without drink

In [26]:
import numpy as np
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

def text_sentiment_flair(text):
  sentence = Sentence(text)
  classifier.predict(sentence)
  if len(sentence.labels)>0:
    return sentence.labels[0].value
  else:
    return str(0)
print(len(news_df['article']))

for i in tqdm(range(0, 100), initial = 0,
              desc ="Running Sentiment Analysis"):
  sentiment=[]
  for index,row in news_df.iterrows():
      article=row['article']
      positive=0
      negative=0
      sentences=article.split('.')
      sentences = pd.Series(list(filter(None, sentences)))
      predictions = sentences.map(lambda x : text_sentiment_flair(x))
      for i in predictions:
          if(i!='0'):
              if(i=='POSITIVE'):
                  positive+=1
              else:
                  negative+=1
      if(positive<negative):
          sentiment.append(-1)
      elif (positive==negative):
          sentiment.append(0)
      else:
        sentiment.append(1)
  news_df['sentiment']=sentiment



2022-06-09 06:35:11,584 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
0


Running Sentiment Analysis: 100%|██████████| 100/100 [00:00<00:00, 5528.13it/s]


In [14]:
import plotly.express as px
fig = px.scatter(data_frame=news_df,y='sentiment',color='sentiment')
fig.show()